In [3]:
!pip install datasets sentencepiece


In [5]:
import pandas as pd
from transformers import AutoTokenizer


In [6]:
!pip install amseg

In [7]:
!pip install pdf2image

In [8]:
!pip install pytesseract

if you want to play with GPT token playground to see how it tokenize words and sentences [GPT token playground](https:/gpt-tokenizer.dev/)



In [14]:

with open('cleaned_labeled_ner_data.conll', 'r') as file:
    lines = file.readlines()

In [15]:

# Process lines as needed
data = [line.strip().split('\t') for line in lines]  # Adjust the split based on your delimiter
df = pd.DataFrame(data)

In [ ]:
df

,0
0,3pcs B-PRODUCT
1,silicon I-PRODUCT
2,brush I-PRODUCT
3,spatulas I-PRODUCT
4,እስከ O
...,...
177806,ያሉበት O
177807,ድረስ O
177808,በነፃ O
177809,እናደርሳለን O


In [20]:
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data if len(item) > 1]  # Extract labels only if item has 2 elements


In [21]:
# data

## Usinng different Amh tokenizers from HuggingFace



In [22]:
# Initialize the tokenizer
model_name = "xlm-roberta-base"  # Change to any approprate model from Hugging Face if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [23]:
# Function to tokenize and align labels
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.tokenize(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [24]:
# Tokenize and align labels
aligned_tokens, aligned_labels = tokenize_and_align_labels(tokenizer, tokens, labels)

In [25]:
# Display the first 10 results
for token, label in zip(aligned_tokens[:20], aligned_labels[:20]):
    print(f"{token:20} {label}")


▁ለማ                  O
ዘ                    O
ዝ                    O
▁84                  O
20                   O
▁ይ                   O
ደው                   O
ሉ                    O
:                    O
▁84                  O
20                   O
▁የሚያ                 O
ምሩ                   O
▁የ                   O
ሴ                    O
ቶች                   O
▁                    O
ባህ                   O
ላዊ                   O
▁አላ                  O


### Using betterTokenizer which google's sentence piece





link for the tokenizer: [link](https://github.com/uhh-lt/amharicprocessor)

In [26]:
from amseg.amharicSegmenter import AmharicSegmenter


In [27]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

In [29]:
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data if len(item) > 1]  # Extract labels only if item has 2 elements


In [30]:
# Function to align tokens with their respective labels
def align_tokens_with_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []

    # Tokenize each word/token with associated label
    for word, label in zip(tokens, labels):
        tokenized_word = tokenizer.amharic_tokenizer(word)  # Tokenize the word
        aligned_tokens.extend(tokenized_word)  # Add tokens to the list

        # Assign the label to the first subtoken and 'O' to subsequent subtokens
        aligned_labels.extend([label] + ['O'] * (len(tokenized_word) - 1))

    return aligned_tokens, aligned_labels

In [31]:
# Example usage with different variable names
new_tokens, new_labels = align_tokens_with_labels(segmenter, tokens, labels)

In [32]:
# Output the first 20 aligned tokens and labels
for token, label in zip(new_tokens[:20], new_labels[:20]):
    print(f"{token}: {label}")


ለማዘዝ: O
8420: O
ይደውሉ: O
:: O
8420: O
የሚያምሩ: O
የሴቶች: O
ባህላዊ: O
አላባሽ: O
በተለያየ: O
ንድፍ: O
ይዘዙ: O
!: O
ለማዘዝ: O
ከታች: O
ያለውን: O
ሊንክ: O
ይጠቀሙ: O
!: O
የሃሎማርኬት: O


In [34]:
# Saving the final tokens and labels to a CSV file
output_df = pd.DataFrame({'Token': new_tokens, 'Label': new_labels})
output_df.to_csv('final_tokens_labels.csv', index=False)



```
# This is formatted as code
```

## Custom Amh Tokenizer

## Amharic SentencePiece Tokenizer

The **Amharic SentencePiece Tokenizer** segments Amharic text into subwords, enhancing the efficiency of natural language processing tasks like Named Entity Recognition (NER). By breaking down words into smaller units, it improves the model's ability to handle rare words and morphological variations.


Dataset used for training the custom tokenizer [dataset link](https://huggingface.co/datasets/israel/Amharic-News-Text-classification-Dataset?row=23)

In [36]:
from datasets import load_dataset
import sentencepiece as spm


In [37]:
#Load the dataset
dataset = load_dataset("israel/Amharic-News-Text-classification-Dataset")

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/150M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/37.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/41186 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10297 [00:00<?, ? examples/s]

In [38]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 41186
    })
    test: Dataset({
        features: ['Unnamed: 0', 'headline', 'category', 'date', 'views', 'article', 'link', 'word_len'],
        num_rows: 10297
    })
})

In [39]:
# Extract the text from the dataset
texts = dataset['train']['article']


In [ ]:
# texts

In [40]:
# Filter out None values and prepare the data for SentencePiece
filtered_texts = [text for text in texts if text is not None]  # Remove None values
text_data = "\n".join(filtered_texts)

In [41]:
# Save to a temporary file
with open('temp_text.txt', 'w', encoding='utf-8') as f:
    f.write(text_data)

In [42]:
# Train SentencePiece model
spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=AmharicSPM --vocab_size=100000')

In [43]:
# spm.SentencePieceTrainer.train('--input=temp_text.txt --model_prefix=am-word --model_type=word  --vocab_size=100000')


In [44]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [45]:
# Tokenize the input text
text = "ቁ.2 ለቡ  መዳህኒዓለም ቤተ/ክርስቲያን ፊት ለፊት  #ዛም_ሞል 2ኛ ፎቅ ቢሮ ቁጥር.214"
tokens = tokenizer.encode(text, out_type=str)  # Use encode method for tokenization
#Print the tokenized words
print(tokens)

['▁ቁ', '.', '2', '▁ለቡ', '▁መዳ', 'ህ', 'ኒ', 'ዓ', 'ለም', '▁ቤተ', '/', 'ክርስቲያን', '▁ፊት', '▁ለፊት', '▁', '#', 'ዛ', 'ም', '_', 'ሞል', '▁2', 'ኛ', '▁ፎቅ', '▁ቢሮ', '▁ቁጥር', '.', '214']


### Using the custom tokenizer

In [47]:
# Process lines: separate tokens and labels
data = [line.strip().split() for line in lines if line.strip()]  # Split based on spaces
tokens = [item[0] for item in data]  # Extract tokens
labels = [item[1] for item in data if len(item) > 1]  # Extract labels only if item has 2 elements


In [48]:
tokenizer = spm.SentencePieceProcessor(model_file='AmharicSPM.model')

In [49]:
def tokenize_and_align_labels(tokenizer, tokens, labels):
    aligned_tokens = []
    aligned_labels = []
    token_ids = []

    for word, label in zip(tokens, labels):
        tokenized_ids = tokenizer.encode(word, out_type=int)  # Get token IDs
        tokenized_words = tokenizer.decode(tokenized_ids).split()  # Decode back to words

        aligned_tokens.extend(tokenized_words)  # Add decoded words to the list
        token_ids.extend(tokenized_ids)  # Add token IDs to the list

        aligned_labels.extend([label] + ['O'] * (len(tokenized_words) - 1))

    return aligned_tokens, token_ids, aligned_labels


In [50]:
# Tokenize and align labels
custom_tokens, custom_ids, custom_labels = tokenize_and_align_labels(tokenizer, tokens, labels)


In [51]:
# Display results from index 50 to 80 with tokenized words, their IDs, and labels
for token, token_id, label in zip(custom_tokens[:20], custom_ids[:20], custom_labels[:20]):
    print(f"{token:<25} {token_id:<10} {label}")


ለማዘዝ                      15         O
8420                      23793      O
ይደውሉ                      6383       O
⁇                         615        O
8420                      567        O
የሚያምሩ                     63344      O
የሴቶች                      0          O
ባህላዊ                      6383       O
አላባሽ                      615        O
በተለያየ                     1808       O
ንድፍ                       5          O
ይዘዙ                       995        O
⁇                         967        O
ለማዘዝ                      1785       O
ከታች                       46853      O
ያለውን                      714        O
ሊንክ                       1640       O
ይጠቀሙ                      33262      O
⁇                         567        O
የ                         46778      O
